In [83]:
from langchain_anthropic import ChatAnthropic

pinecone_namespace = "simple-rag-1"
chat_history = [{
    "role": "human",
    "message": "Describe the hierarchy and organizational structure of the judicial system in Pakistan."
},
    {
    "role": "ai",
    "message": "The judicial system in Pakistan follows a hierarchical structure with the Supreme Court at the top, followed by High Courts in each province, and subordinate courts which include district and sessions courts, civil and judicial magistrates."
},
    {
    "role": "human",
    "message": "what is the role of subordinate courts"
},
    {
    "role": "ai",
    "message": "Subordinate courts handle the bulk of civil and criminal cases at the district level."
}

]
question = "what challenges are faced by them"

model = ChatAnthropic(model="claude-3-sonnet-20240229")

test_response = model.invoke("Hello how are you?")

### Rephrase Query According to Conversation

In [84]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

rephrase_query_prompt = [
    SystemMessage(
        "Given the following conversation, repharase the followup question to be a standalone question, that is understandable and optimized for similarity search while preserving the meaning. The output should just be the rephrased question")
] + [
    HumanMessage(entry["message"]) if entry["role"] == "human" else AIMessage(entry["message"]) for entry in chat_history
] + [HumanMessage(f"followup: \n {question}")]
rephrase_query_answer = model.invoke(rephrase_query_prompt)
query = rephrase_query_answer.content
print(query)

What are the major challenges faced by subordinate courts in Pakistan's judicial system?


### Find relevant chunks to rephrase query

In [85]:
from utils import get_relevant_chunks

relevant_chunks = get_relevant_chunks(query, pinecone_namespace, 10)
relevant_chunks_text = [item["metadata"]["text"] for item in relevant_chunks]

### Rerank chunks

In [86]:
from utils import rerank_chunks, print_chunks

rerank_results = rerank_chunks(query, relevant_chunks_text)
most_relevant_chunks = rerank_results[0:3]
most_relevant_chunks_text = "----------------------\n".join([c["text"] for c in most_relevant_chunks])
print(most_relevant_chunks_text)

‫המקדיש‪ ,‬כיוון שעל ידי כך לא יוכל לפעול ביעילות מי שאמור למלא את משימת‬
‫החלוקה של הכספים‪ .‬העברת הנכסים אל הממונה נועדה אפוא לאפשר לו למלא את‬
‫המשימה שנקבעה על־ידי המקדיש‪.‬‬
‫‪38‬‬
‫אולם במקרה זה‪ ,‬כשלא הוגדר נהנה מסוים‪ ,‬לא יימצא אדם הרשאי לטעון‬
‫שהוא הנהנה‪ .‬מי יהיה זכאי לאכוף על הנאמן למלא את התחייבותו ולהימנע‬
‫מלהפיק תועלת לעצמו משליטתו בנכסי הנאמנות?‬
‫באנגליה אימצו את דרך הפתרון הרומי‪ ,‬היינו‪ ,‬בעזרת סמכות האכיפה של‬
‫רשות שלטונית‪ ,‬אשר ‪ -‬כמו הפרטור הרומי ‪ -‬עסקה גם בפתרון סכסוכים בין‬
‫אזרחים על יסוד שיקולים של עשיית צדק ויושר‪ .‬בראש מערכת זו עמד ה‪-‬‬
‫‪ ,Chancellor‬שבמשך מאות שנים מילא תפקיד ביצועי מרכזי בממלכה‪ .‬רק‬
‫בראשית המאה הי״ט הפכה מערכת הטיפול בפניות הנתינים למסגרת שיפוטית‪.‬‬
‫סמכות זו מיושמת כיום במסגרת מחלקת ^‪ Chancery-‬של בית‪-‬המשפט‪.‬‬
‫‪39‬‬
‫"קופה של צדקה״ בלשון התלמוד‪ ,‬ראו בבא בתרא ח‪ ,‬ע״ב‪.‬‬
‫״תמחוי״ בלשון התלמוד‪ .‬ראו בבא בתרא ח‪ ,‬ע״ב‪.‬‬
‫בשיטות משפטיות ללא מסגרת פורמלית של רישום בעלות בנכסים אין מייחסים חשיבות לנושא‬-------------

### Prompt LLM for an Actual Answer

In [82]:
get_answer_prompt = [
    SystemMessage("You are a legal assistant, you are given some knowledge about the topic, and a question. You must answer user's question only if its evident from the knowledge, If you cannot find or extract enough knowledge, let the user know that as per your limited knowledge you are not sure and won't be able to answer them"),
    HumanMessage(f"Knowledge is made up of top text chunks in order, acquired after performing a similarity search in the knwoledge base for user's query \n Knowledge: \n {most_relevant_chunks_text} \n question: {query}")
]
get_answer_res = model.invoke(get_answer_prompt)
print(get_answer_res.content)

Based on the given knowledge, I do not have enough information to confidently answer your question about the major challenges and issues faced by subordinate courts in Pakistan in effectively carrying out their functions. The provided text appears to be in a different language and does not seem directly relevant to the court system in Pakistan. Without additional relevant knowledge on this specific topic, I cannot provide a substantive response. I apologize that I cannot be more helpful in answering your query with my limited knowledge.
